# Parameterized evolution

In [54]:
import numpy as np

matrix = np.diag([1, 2, 5, 0])

In [40]:
from qiskit.opflow import MatrixOp

op = MatrixOp(matrix)

print(op)

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 3.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


In [41]:
op.exp_i()
print(type(op.exp_i()))
print(type(op.exp_i().primitive))

<class 'qiskit.opflow.primitive_ops.circuit_op.CircuitOp'>
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>


In [42]:
circuit = op.exp_i().primitive

In [43]:
from qiskit import transpile
basis_gates = ['cx', 'rx', 'ry', 'rz', 'x', 'h', 'u']
unrolled = transpile(circuit, basis_gates=basis_gates, optimization_level=0)
print(unrolled.draw())

global phase: -4.0708
     ┌──────────────────┐           ┌─────────────────┐           »
q_0: ┤ U(0.0,3π/4,3π/4) ├──■────────┤ U(π,3π/4,-3π/4) ├────────■──»
     ├─────────────────┬┘┌─┴─┐┌─────┴─────────────────┴─────┐┌─┴─┐»
q_1: ┤ U(π/2,-π/2,π/2) ├─┤ X ├┤ U(π/2,3.57079632679490,π/2) ├┤ X ├»
     └─────────────────┘ └───┘└─────────────────────────────┘└───┘»
«     ┌─────────────────────────────────────────┐
«q_0: ┤ U(π,4.42699081698724,0.285398163397448) ├
«     └────────────┬────────────────┬───────────┘
«q_1: ─────────────┤ U(0.0,π/2,π/2) ├────────────
«                  └────────────────┘            


In [46]:
from qiskit.circuit import Parameter
from qiskit.opflow import QFI, StateFn

time = Parameter('t')

evolved_circuit = (time * op).exp_i().primitive
unrolled = transpile(evolved_circuit, basis_gates=basis_gates, optimization_level=0)

state = StateFn(unrolled)
qfi = QFI().convert(state, [time])



QiskitError: 'Error decomposing node hamiltonian: Unable to generate Unitary matrix for unbound t parameter 1.0*t'

In [23]:
from qiskit.opflow import I, Z

op = (Z ^ I) + 2 * (I ^ Z) 

print(op)
print(type(op))
print(type(op.primitive))

1.0 * ZI
+ 2.0 * IZ
<class 'qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp'>
<class 'qiskit.quantum_info.operators.symplectic.sparse_pauli_op.SparsePauliOp'>


In [25]:
from qiskit.opflow import PauliTrotterEvolution

trotter = PauliTrotterEvolution()
evolved = trotter.convert(op.exp_i()).to_circuit()
print(evolved)

     ┌───────┐
q_0: ┤ RZ(4) ├
     ├───────┤
q_1: ┤ RZ(2) ├
     └───────┘


## Add parameterized time

In [26]:
from qiskit.circuit import Parameter

time = Parameter('t')

In [27]:
trotter = PauliTrotterEvolution()
op = time * op
print(op)

1.0*t * (
  1.0 * ZI
  + 2.0 * IZ
)


In [35]:
evolved_circuit = trotter.convert(op.exp_i()).to_circuit()
print(evolved)

global phase: 0
     ┌───────────┐
q_0: ┤ RZ(4.0*t) ├
     ├───────────┤
q_1: ┤ RZ(2.0*t) ├
     └───────────┘


In [36]:
from qiskit.opflow import QFI, StateFn

state = StateFn(evolved_circuit)
qfi = QFI().convert(state, [time])

In [38]:
qfi.bind_parameters({time: 1}).eval()

array([[-7.10542736e-15+1.3484e-14j]])

In [55]:
matrix_op = MatrixOp(matrix)
print(matrix_op)

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


In [56]:
pauli_op = matrix_op.to_pauli_op()
pauli_op *= time

In [58]:
evolved_circuit = trotter.convert(pauli_op.exp_i()).to_circuit()
print(evolved_circuit)

global phase: -2.0*t
     ┌───┐┌────────────┐┌───┐┌───────────┐ 
q_0: ┤ X ├┤ RZ(-3.0*t) ├┤ X ├┤ RZ(2.0*t) ├─
     └─┬─┘└────────────┘└─┬─┘├───────────┴┐
q_1: ──■──────────────────■──┤ RZ(-1.0*t) ├
                             └────────────┘


In [59]:
from qiskit.opflow import QFI, StateFn

state = StateFn(evolved_circuit)
qfi = QFI().convert(state, [time])

In [60]:
qfi.bind_parameters({time: 1}).eval()

array([[-5.32907052e-15+1.144e-15j]])

# Primitive operators

In [72]:
from qiskit import QuantumCircuit
from qiskit.opflow import PrimitiveOp

circuit = QuantumCircuit(1)

op = PrimitiveOp(circuit)
print(type(op))
print(type(op.primitive))

<class 'qiskit.opflow.primitive_ops.circuit_op.CircuitOp'>
<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>


In [70]:
op = PrimitiveOp(matrix)
print(type(op))
print(type(op.primitive))

<class 'qiskit.opflow.primitive_ops.matrix_op.MatrixOp'>
<class 'qiskit.quantum_info.operators.operator.Operator'>


In [71]:
op = Z ^ I
print(type(op))
print(type(op.primitive))

<class 'qiskit.opflow.primitive_ops.pauli_op.PauliOp'>
<class 'qiskit.quantum_info.operators.symplectic.pauli.Pauli'>


## List operators

In [73]:
from qiskit.opflow import ListOp, X

op = ListOp([Z, I, Z, X])
print(op)

ListOp([
  Z,
  I,
  Z,
  X
])


In [74]:
from qiskit.opflow import One, Zero

(One.adjoint().compose(op).compose(Zero)).eval()

[0j, 0j, 0j, (1+0j)]

ComposedOp, TensoredOp, SummedOp

In [75]:
pauli_op = Z ^ I
matrix_op = MatrixOp(matrix)

In [77]:
from qiskit.opflow import SummedOp

SummedOp([pauli_op, matrix_op])

SummedOp([PauliOp(Pauli('ZI'), coeff=1.0), MatrixOp(Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2)), coeff=1.0)], coeff=1.0, abelian=False)

In [78]:
pauli_op + matrix_op

SummedOp([PauliOp(Pauli('ZI'), coeff=1.0), MatrixOp(Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2)), coeff=1.0)], coeff=1.0, abelian=False)

In [80]:
state = Zero ^ 2
state.adjoint().compose(pauli_op + matrix_op).compose(state).eval()

(2+0j)

In [81]:
(~state @ (pauli_op + matrix_op) @ state).eval()

(2+0j)

# StateFns

In [82]:
from qiskit.opflow import StateFn, VectorStateFn

vector = np.array([1, 0, 0, 0])
state = StateFn(vector)
print(state)

VectorStateFn(Statevector([1.+0.j 0.+0.j 0.+0.j 0.+0.j]))


In [83]:
state = VectorStateFn(vector)
print(state)

VectorStateFn(Statevector([1.+0.j 0.+0.j 0.+0.j 0.+0.j]))


In [84]:
vector = np.array([10, 0, 0, 9])
state = StateFn(vector)
print(state)

VectorStateFn(Statevector([10.+0.j  0.+0.j  0.+0.j  9.+0.j]))


In [85]:
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

state = StateFn(circuit)
print(state)

CircuitStateFn(
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
)


In [86]:
state.to_matrix_op()

VectorStateFn(Statevector([0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]), coeff=1.0, is_measurement=False)

In [87]:
state.eval()

VectorStateFn(Statevector([0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]), coeff=1.0, is_measurement=False)

In [94]:
amplitudes = {'00': np.sqrt(0.5), '11': np.sqrt(0.5)}
state = StateFn(amplitudes)

print(state)

DictStateFn({'00': 0.7071067811865476, '11': 0.7071067811865476})


In [95]:
state.eval()

VectorStateFn(Statevector([0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]), coeff=1.0, is_measurement=False)

In [99]:
print((~StateFn(Z ^ I) @ state).eval())
print((~state @ (Z ^ I) @ state).eval())

0j
0j


In [101]:
print(~StateFn(Z ^ I))

OperatorMeasurement(ZI)
